In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [49]:
df = pd.read_csv("https://raw.githubusercontent.com/berdikhanova/undergraduate-thesis/main/the-thesis/Final_Data/Data_Astana_DiD.csv")

In [50]:
df

,Region_ID,Region,Year,Treatment,Total-City,Total(KAZ)-City,Total(RUS)-City,Total(Low)-City,Total(Low)-City %,KAZ(Low)-City,...,Total_Low,Total_Low-Mid,Total_Mid,Total_High-Mid,Total_High,Low_%,Low-Mid%,Mid_%,High_Mid%,High_%
0,2,Aktobe,2014,0,2649,1771,878,124,0.05,88,...,268,975,1295,779,1191,5.94%,21.63%,28.73%,17.28%,26.42%
1,5,Astana City,2014,0,3251,1598,1653,140,0.04,83,...,140,516,936,683,976,4.31%,15.87%,28.79%,21.01%,30.02%
2,9,Karaganda,2014,0,3581,1532,2049,139,0.04,94,...,360,1560,1844,741,589,7.07%,30.62%,36.20%,14.55%,11.56%
3,2,Aktobe,2015,0,2766,2019,747,181,0.07,133,...,339,1011,1295,921,1054,7.34%,21.88%,28.03%,19.94%,22.81%
4,5,Astana City,2015,0,3398,1885,1513,178,0.05,119,...,178,568,966,686,1000,5.24%,16.72%,28.43%,20.19%,29.43%
5,9,Karaganda,2015,0,3526,1695,1831,107,0.03,57,...,255,1326,1848,855,693,5.12%,26.64%,37.13%,17.18%,13.92%
6,2,Aktobe,2016,0,2680,1928,752,169,0.06,130,...,329,1025,1365,745,1134,7.16%,22.29%,29.69%,16.20%,24.66%
7,5,Astana City,2016,0,3782,2088,1694,150,0.04,82,...,150,563,1007,752,1310,3.97%,14.89%,26.63%,19.88%,34.64%
8,9,Karaganda,2016,0,3090,1573,1517,65,0.02,33,...,175,1051,1720,776,753,3.91%,23.49%,38.44%,17.34%,16.83%
9,2,Aktobe,2017,0,2808,2119,689,336,0.12,248,...,616,1051,1895,1267,41,12.71%,21.69%,39.11%,26.15%,0.85%


In [42]:
df.drop(df.columns.difference(['Region',"Year",'Region_ID',
'Region',
'Year',
'Treatment',
'Total(Low)-City %',
'Population_City',
'New_Schools',
'Number_of_Students_City',
'Number_of_Teachers',
'Students_per_School_City',
'Students_Per_Teacher',
'Teachers_per_School']), 1, inplace=True)
df.head()

/var/folders/m8/s9x7jcmj1hg3y0p_lg7dx9z00000gn/T/ipykernel_17346/1112487293.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['Region',"Year",'Region_ID',


,Region_ID,Region,Year,Treatment,Total(Low)-City %,Population_City,New_Schools,Number_of_Teachers,Students_per_School_City,Students_Per_Teacher,Teachers_per_School
0,2,Aktobe,2014,0,0.05,498794,6,16023.00,683,7,36
1,5,Astana City,2014,0,0.04,814435,9,7092.00,1071,14,77
2,9,Karaganda,2014,0,0.04,1077482,2,21091.00,562,8,37
3,2,Aktobe,2015,0,0.07,510568,5,16128.51,707,8,36
4,5,Astana City,2015,0,0.05,852882,4,7138.70,1146,15,74


In [43]:
df['Time'] = np.where(df['Year'] >= 2019, 1,0 )
df['Treated'] = np.where(df['Region'] >= 'Astana City' , 1,0 )
df['DiD'] = df['Time'] * df['Treated']

In [44]:
df = df[df["Region"].isin(["Astana City","Karaganda"])]
df

,Region_ID,Region,Year,Treatment,Total(Low)-City %,Population_City,New_Schools,Number_of_Teachers,Students_per_School_City,Students_Per_Teacher,Teachers_per_School,Time,Treated,DiD
1,5,Astana City,2014,0,0.04,814435,9,7092.00,1071,14,77,0,1,0
2,9,Karaganda,2014,0,0.04,1077482,2,21091.00,562,8,37,0,1,0
4,5,Astana City,2015,0,0.05,852882,4,7138.70,1146,15,74,0,1,0
5,9,Karaganda,2015,0,0.03,1088300,1,21229.88,605,8,38,0,1,0
7,5,Astana City,2016,0,0.04,872619,3,7251.00,1224,17,73,0,1,0
8,9,Karaganda,2016,0,0.02,1096792,4,21662.00,592,8,39,0,1,0
10,5,Astana City,2017,0,0.15,972692,1,8388.00,1292,16,80,0,1,0
11,9,Karaganda,2017,0,0.12,1098499,1,20689.00,614,9,37,0,1,0
13,5,Astana City,2018,1,0.20,1030577,4,9290.00,1419,16,88,0,1,0
14,9,Karaganda,2018,0,0.15,1099146,2,20770.00,624,9,38,0,1,0


In [45]:
x = df.iloc[:,5:18]
y = df.iloc[:,4]

In [47]:
model = LinearRegression().fit(x, y)
model.coef_

array([ 1.67902072e-06,  1.35857504e-02, -4.30959216e-05, -1.17249895e-04,
        1.44602452e-02, -7.98513444e-03,  5.44715658e-02,  0.00000000e+00,
        5.44715658e-02])

In [48]:
X2 = sm.add_constant(x)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:      Total(Low)-City %   R-squared:                       0.840
Model:                            OLS   Adj. R-squared:                  0.700
Method:                 Least Squares   F-statistic:                     5.990
Date:                Fri, 09 Dec 2022   Prob (F-statistic):             0.0110
Time:                        23:24:36   Log-Likelihood:                 30.180
No. Observations:                  16   AIC:                            -44.36
Df Residuals:                       8   BIC:                            -38.18
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Population_City         

/Users/marinaberdikhanova/Library/Python/3.8/lib/python/site-packages/scipy/stats/_stats_py.py:1772: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
